In [1]:
!pip install --user numba

  Using cached llvmlite-0.34.0.tar.gz (107 kB)
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [26 lines of output]
      running bdist_wheel
      /cvmfs/sft.cern.ch/lcg/views/LCG_102swan/x86_64-centos7-gcc11-opt/bin/python /tmp/pip-install-81mrm7v9/llvmlite_2e733283a24c48df99f64851c8a757c7/ffi/build.py
      LLVM version... Traceback (most recent call last):
        File "/tmp/pip-install-81mrm7v9/llvmlite_2e733283a24c48df99f64851c8a757c7/ffi/build.py", line 105, in main_posix
          out = subprocess.check_output([llvm_config, '--version'])
        File "/cvmfs/sft.cern.ch/lcg/releases/Python/3.9.12-9a1bc/x86_64-centos7-gcc11-opt/lib/python3.9/subprocess.py", line 424, in check_output
          return run(*popenargs, stdout=PIPE, timeout=timeout, check=True,
        File "/cvmfs/sft.cern.ch/lcg/releases/Python/3.9.12-9a1bc/x86_64-centos7-gcc11-opt/lib/python3.9/subpro

In [2]:
import matplotlib.pyplot as plt
import os
import numpy as np
import pandas as pd

In [3]:
input_folder = "/eos/user/v/vdurupt/reco_comparison/supercluster_regression/electrons"
output_folder = "/eos/user/v/vdurupt/reco_comparison/RecoPlots/RecoComparison_regression_UL18_SCregression"
os.makedirs(output_folder, exist_ok=True)

In [4]:
reco = "DeepSC_AlgoD"
df_1 = pd.HDFStore(f"{input_folder}/ele_UL18_123X_Mustache_object.h5py", "r")["df"]
df_2 = pd.HDFStore(f"{input_folder}/ele_UL18_123X_{reco}_object.h5py", "r")["df"]
df_1.rename(columns={"output_object.csv":"calomatched"}, inplace=True)
df_2.rename(columns={"output_object.csv":"calomatched"}, inplace=True)
df_1.rename(columns={"ncsl_tot":"ncls_tot"}, inplace=True)
df_2.rename(columns={"ncsl_tot":"ncls_tot"}, inplace=True)
df_1 = df_1.iloc[:-1]
df_2 = df_2.iloc[:-1]

df_1["Eraw_ov_Esim"] = df_1.en_sc_raw / df_1.calo_en_sim
df_1["Ecorr_ov_Esim"] = df_1.en_sc_calib / df_1.calo_en_sim
df_2["Eraw_ov_Esim"] = df_2.en_sc_raw / df_2.calo_en_sim
df_2["Ecorr_ov_Esim"] = df_2.en_sc_calib / df_2.calo_en_sim

df_1["Eraw_ov_EGen"] = df_1.en_sc_raw / df_1.genpart_en
df_1["Ecorr_ov_EGen"] = df_1.en_sc_calib / df_1.genpart_en
df_2["Eraw_ov_EGen"] = df_2.en_sc_raw / df_2.genpart_en
df_2["Ecorr_ov_EGen"] = df_2.en_sc_calib / df_2.genpart_en

df_1.rename(columns={"sc_index": "old_index"}, inplace=True)
df_2.rename(columns={"sc_index": "new_index"}, inplace=True)


df_join = df_1.merge(df_2, on = ["runId","eventId","caloindex"], suffixes = ["_old", "_new"], indicator = True)

In [5]:
#Resolution

In [6]:
def bin_analysis(col):
    def f(df):
        m = df[col].mean()
        A = (df[col].quantile(0.84) - df[col].quantile(0.16))/2
        B = (df[col].quantile(0.975) - df[col].quantile(0.025))/2
        return pd.Series({
            "m": m,
            "w68": A,
            "w95": B,
            "N": df[col].count()
        })
    return f

def bin_analysis_details(col):
    def f(df):
        m = df[col].mean()
        qu = df[col].quantile(0.84)
        qd = df[col].quantile(0.16)
        A = (qu - qd)/2
        quu = df[col].quantile(0.975)
        qdd = df[col].quantile(0.025)
        B = (quu- qdd )/2
        return pd.Series({
            "m": m,
            "w68": A,
            "w95": B,
            "w68_u": qu,
            "w68_d": qd,
            "w95_u" : quu,
            "w95_d" : qdd,
            "N": df[col].count()
        })
    return f


def get_quantiles(df):
    return df.quantile(0.025), df.quantile(0.16), df.quantile(0.5), df.quantile(0.84), df.quantile(0.975)


def movingaverage(interval, window_size):
    window= np.ones(int(window_size))/float(window_size)
    return np.convolve(interval, window, 'same')

from numba import jit

@jit(nopython=True)
def get_central_smallest_interval(df, xrange, nbins, Ntrial=10000, perc=0.68):
    H = np.histogram(df, bins=nbins, range=xrange)
    
    #xx = movingaverage(H[0], 10)
    
    xmax = H[1][np.argmax(H[0])]
    
    #xmax = np.mean(df[(df>=xrange[0]) & (df<xrange[1])])
    deltax = (xrange[1]-xrange[0])/(2*Ntrial)
    
    N = df.size
    xd = xmax-deltax
    xu = xmax+deltax
    for i in range(Ntrial):
        q = np.sum((df>xd) &(df<xu))/ N
        if q>=perc: 
            break
        xd = xd-deltax
        xu = xu+deltax
    return xmax, xd, xu

def bin_analysis_central_smallest(col, xrange=(0.6, 1.2), nbins=200, Ntrial =10000):
    def f(df):
        data = df[col]
        xmax, qd, qu = get_central_smallest_interval(data.values, xrange=xrange, nbins=nbins, Ntrial =Ntrial )
        return pd.Series({
            "m": xmax,
            "w68": (qu-qd)/2,
            "w68_u": qu,
            "w68_d": qd,
            "N": df[col].count()
        })
    return f

In [7]:
#Dynamic fit function

In [8]:
from numba import jit 

@jit
def cruijff(x, A, m, sigmaL,sigmaR, alphaL, alphaR):
    dx = (x-m)
    SL = np.full(x.shape, sigmaL)
    SR = np.full(x.shape, sigmaR)
    AL = np.full(x.shape, alphaL)
    AR = np.full(x.shape, alphaR)
    sigma = np.where(dx<0, SL,SR)
    alpha = np.where(dx<0, AL,AR)
    f = 2*sigma*sigma + alpha*dx*dx ;
    return A* np.exp(-dx*dx/f) 

from scipy.optimize import curve_fit

def fit_cruijff(data, bins, xrange):
    H= np.histogram(data,bins=bins, range=xrange)
    x = H[1][:-1]
    Y = H[0]
    # delete empty bins
    x = x[Y>0]
    Y =Y[Y>0]
    params, pcovs = curve_fit(cruijff, x, Y, p0=[np.max(Y), 1, 0.02, 0.02,  0.15,0.15], 
                              sigma=np.sqrt(Y),absolute_sigma=True)
    return params, pcovs
    
    
def bin_analysis_cruijff(col, nbins=300, prange=0.95):
    def f(df):
        data = df[col].values
        # use the mode looking function to get a meaninful interval centered around the maximum and no tails
        # we just need a rought center estimate
        mean, xd, xu = get_central_smallest_interval(data, xrange=(0.6,1.3), nbins=200, perc=prange)
        
        try:
            params, pcovs = fit_cruijff(data, bins=nbins, xrange=(xd, xu) )
            perr = np.sqrt(np.diag(pcovs))

            return pd.Series({
                "m": params[1],
                "sigmaL": params[2], 
                "sigmaR": params[3],
                "alphaL": params[4],
                "alphaR": params[5],
                "m_err": perr[1],
                "sigmaL_err": perr[2], 
                "sigmaR_err": perr[3],
                "alphaL_err": perr[4],
                "alphaR_err": perr[5],
                "A": params[0],
                "A_err": perr[0],
                "N": df[col].count(),
                "xmin": xd,
                "xmax": xu
            })
        except:
            print("Fit failed")
            return pd.Series({
                "m": mean,
                "sigmaL": -1, 
                "sigmaR": -1,
                "alphaL": -1,
                "alphaR": -1,
                "m_err": -1,
                "sigmaL_err": -1, 
                "sigmaR_err": -1,
                "alphaL_err": -1,
                "alphaR_err": -1,
                "A": -1,
                "A_err": -1,
                "N": df[col].count(),
                "xmin": xd,
                "xmax": xu
            })
    return f

In [9]:
 #Test the fit

In [10]:
df_1[:]["caloindex"]

0        0.0
1        2.0
2        3.0
3        0.0
4        1.0
        ... 
15893    3.0
15894    2.0
15895    1.0
15896    3.0
15897    0.0
Name: caloindex, Length: 15898, dtype: float64

In [11]:
df_2[:]["caloindex"]

0       -999.0
1          0.0
2       -999.0
3          1.0
4          2.0
         ...  
15896      0.0
15897      1.0
15898      2.0
15899      0.0
15900      3.0
Name: caloindex, Length: 15901, dtype: float64

In [12]:
df_join

,calomatched_old,sc_matched_old,calomatched_old,caloindex,genmatched_old,genindex_old,old_index,seed_index_old,en_seed_old,et_seed_old,...,gen_pt_new,nVtx_new,rho_new,obsPU_new,truePU_new,Eraw_ov_Esim_new,Ecorr_ov_Esim_new,Eraw_ov_EGen_new,Ecorr_ov_EGen_new,_merge


In [13]:
##Test the fit

In [18]:
bins= 300
var = "Ecorr_ov_EGen"
emin = 10
emax = 100
eta_min = 1
eta_max = 1.479


dfx_must = df_join[(df_join.seed_eta_old.abs()>eta_min)&(df_join.seed_eta_old.abs()<eta_max)&(df_join.et_seed_old>=emin)&(df_join.et_seed_old<emax)][var+"_old"]
dfx_deep = df_join[(df_join.seed_eta_new.abs()>eta_min)&(df_join.seed_eta_new.abs()<eta_max)&(df_join.et_seed_new>=emin)&(df_join.et_seed_new<emax)][var+"_new"]
    
mean, xd, xu = get_central_smallest_interval(dfx_deep.values, xrange=(0.5, 1.5), nbins=250, perc=0.95)
    
xrange = (xd,xu)
H= np.histogram(dfx_deep,bins=bins, range=xrange)

x = H[1][:-1]
Y = H[0]

x = x[Y>0]
Y =Y[Y>0]


NameError: name 'xd' is not defined

NameError: name 'H' is not defined